In [ ]:
from standard_functions import *
path = os.getcwd()
data_path = os.path.join(os.path.dirname(path), 'Data') 

BloombergData = pd.read_csv(data_path + "/BloombergData_Swap_Features.csv")
preData = pd.read_csv(data_path + "/TestData_Swap_Features_pre.csv")
postData = pd.read_csv(data_path + "/TestData_Swap_Features_post.csv")

insample = np.array(BloombergData.iloc[:,2:].reset_index(drop=True)) 
insample_scaled = [x/100 for x in insample]
insample_tensor = torch.from_numpy(np.float32(insample_scaled))

X_train1, X_val1 = train_validation_split(BloombergData.reset_index(drop=True), 0.1) 
X_train2  = np.array(X_train1.iloc[:,2:].reset_index(drop=True)) 
X_train_scaled = [x/100 for x in X_train2]
X_train_tensor = torch.from_numpy(np.float32(X_train_scaled))

X_val2  = np.array(X_val1.iloc[:,2:].reset_index(drop=True)) 
X_val_scaled = [x/100 for x in X_val2]
X_val_tensor = torch.from_numpy(np.float32(X_val_scaled))

preX_test2  = np.array(preData.iloc[:,2:].reset_index(drop=True)) 
preX_test_scaled = [x/100 for x in preX_test2]
preX_test_tensor = torch.from_numpy(np.float32(preX_test_scaled))

postX_test2  = np.array(postData.iloc[:,2:].reset_index(drop=True)) 
postX_test_scaled = [x/100 for x in postX_test2]
postX_test_tensor = torch.from_numpy(np.float32(postX_test_scaled))


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(8, 2, bias = False)
        )

        self.decoder = nn.Sequential(
            nn.Linear(3, 10, bias = False),
            centered_softmax(),
            nn.Linear(10, 1, bias = False)
        )

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)

    def forward(self, x):

        # Encoder
        x = self.encoder(x)  #Nx2 

        # Maturity 
        mat = torch.arange(1, 31, dtype=torch.float32).repeat(len(x)) # length N*30 array of 1-30,1-30 and so on N times 

        x1 = x.unsqueeze(1).expand(-1, 30, -1).reshape(-1, 2)         # Nx30x2 then reshaped to N*30x2 "reshape(-1, 3)" 3 is for 3 factor. 
                                                                      # If 2-factor, then .reshape(-1, 2)
        result = torch.cat([x1, mat.unsqueeze(1)], dim=1)             # N*30x3
                                                                        
        # Decoder --> gives y for all maturities 
        y = self.decoder(result).reshape(len(x), 30)

        # Transformations: y->p->s: 
        p = torch.exp(-mat.reshape(y.shape[0],30) * y)

        p_cumsum = torch.cumsum(p, dim=1)  # N, 30

        s = (1 - p) / p_cumsum  # N, 30
        
        return s, p, y 

In [ ]:
# torch.manual_seed(1)


# # Initialize autoencoder, MSE loss, and optimizer
# model = Autoencoder()   
# model = Autoencoder()   
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# lr_sched = LR_Scheduler(optimizer, percentage = 0.9, interval = 50)

# # Training
# num_epochs = 3336
# batch_size = 32
# data_loader = torch.utils.data.DataLoader(X_train_tensor, batch_size=batch_size, shuffle=True)

# swap_mats = [1, 2, 3, 5, 10, 15, 20, 30]
# swap_mats0 = [i-1 for i in swap_mats]

# losses_list = []
# vallosses_list = []

# for epoch in range(num_epochs):
#     for batch in data_loader:
        
#         # Forward pass
#         reconstructed, p_final, y_final = model(batch)

#         s_final = reconstructed[:, swap_mats0]
        
#         loss = criterion(s_final, batch)
        
#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward(retain_graph=True)
#         optimizer.step()
#     lr_sched.step()

#     N = len(X_val_tensor)    
#     reconstructed, p_final, y_final = model(X_val_tensor)
#     s_final = reconstructed[:, swap_mats0]
#     loss_val = criterion(s_final, X_val_tensor)

#     N = len(X_train_tensor)    
#     reconstructed, p_final, y_final = model(X_train_tensor)
#     s_final = reconstructed[:, swap_mats0]
#     loss_train = criterion(s_final, X_train_tensor)

#         # append relevant data
#     vallosses_list.append(loss_val.item())
#     losses_list.append(loss_train.item())
        
#     if (epoch + 1) % 100 == 0:
#         print(f"Epoch [{epoch + 1}/{num_epochs}], Loss train: {loss_train.item():.8f}")

In [ ]:
name = "2factor_FIAE"
# torch.save(model.state_dict(), f"models/{name}.pth")
# print(f"Model saved successfully as models/{name}.pth")
model2 = Autoencoder()       # Ensure Autoencoder class is defined
model2.load_state_dict(torch.load(f"models/{name}.pth"))